In [1]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import diffprivlib.models as dp
from sklearn.linear_model import LogisticRegression
from torch import nn, optim
from opacus import PrivacyEngine
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

## Load Training and Testing Data

In [2]:
# Load the data
X_train = pd.read_csv('../../data/X_train.csv', index_col=0)
X_test = pd.read_csv('../../data/X_test.csv', index_col=0)
y_train = pd.read_csv('../../data/y_train.csv', index_col=0)
y_test = pd.read_csv('../../data/y_test.csv', index_col=0)

## Transform Data for the Model

In [3]:
# Check the shapes of the data
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5748, 19), (1437, 19), (5748, 1), (1437, 1))

In [4]:
y_train.head()

,two_year_recid
5355,1
3592,0
29,1
2414,1
5619,1


In [5]:
# Check type of y_train
type(y_train)

pandas.core.frame.DataFrame

In [6]:
# y_train.set_index('Unnamed: 0', inplace=True)
y_train = y_train.squeeze()

# y_test.set_index('Unnamed: 0', inplace=True)
y_test = y_test.squeeze()

In [7]:
# Remove the name of the index
y_train.index.name = None
y_test.index.name = None

In [8]:
# Check type of y_test
type(y_test)

pandas.core.series.Series

In [9]:
y_train.head()

5355    1
3592    0
29      1
2414    1
5619    1
Name: two_year_recid, dtype: int64

In [10]:
y_test.head()

3600    0
4218    0
900     0
5345    1
5682    0
Name: two_year_recid, dtype: int64

In [11]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the training data using the scaler
X_train_scaled = scaler.fit_transform(X_train.drop(['id'], axis=1))
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.drop(['id'], axis=1).columns, index=X_train.index)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test.drop(['id'], axis=1))
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_train.drop(['id'], axis=1).columns, index=X_test.index)

In [12]:
# Check the shapes of the scaled data
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

((5748, 18), (1437, 18), (5748,), (1437,))

In [13]:
# Convert Pandas DataFrames to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

## Defining a Multi-Layer Perceptron (MLP)

In [14]:
# Set the random seed for reproducibility
torch.manual_seed(10000)

# Define the MLP model
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2,hidden_size3,output_size):
        super(MLPModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size3, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        x = self.output_layer(x)
        x = self.sigmoid(x)
        return x

In [15]:
# Initialize the model, loss function, and optimizer
input_size = X_train_scaled.shape[1]
hidden_size1 = 8
hidden_size2 = 8
hidden_size3 = 4
output_size = 1

In [16]:
model_file_path = "../../model/mlp_model.pth"

# Load the saved model
model = MLPModel(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)
model.load_state_dict(torch.load(model_file_path))

<All keys matched successfully>

In [17]:
# Binary Cross Entropy Loss for binary classification
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Convert data to PyTorch DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## Comparing Accuracy of Baseline models with Differentially Private models

### 1. Baseline Logistic Regression Model

In [19]:
clf = LogisticRegression(solver="lbfgs", max_iter=1000)
clf.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [20]:
baseline_lr = clf.score(X_test_scaled, y_test)
print("Non-private LR model test accuracy: %.2f%%" % (baseline_lr * 100))

Non-private LR model test accuracy: 69.66%


### 2. Differentially Private Logistic Regression Model

In [21]:
dp_clf = dp.LogisticRegression(random_state=0, epsilon=8.0)
dp_clf.fit(X_train_scaled, y_train)

/Users/keshvigupta/opt/anaconda3/lib/python3.9/site-packages/diffprivlib/models/logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "


LogisticRegression(accountant=BudgetAccountant(spent_budget=[(8.0, 0)]),
                   data_norm=30.479211207303948, epsilon=8.0, random_state=0)

In [22]:
dp_lr = dp_clf.score(X_test_scaled, y_test)
print("Differentially private LR model test accuracy (epsilon=%.2f): %.2f%%" % 
     (dp_clf.epsilon, (dp_lr * 100)))

Differentially private LR model test accuracy (epsilon=8.00): 67.29%


In [21]:
# Decrease epsilon for increased privacy
dp2_clf = dp.LogisticRegression(random_state=0, epsilon=4.0)
dp2_clf.fit(X_train_scaled, y_train)

/Users/keshvigupta/opt/anaconda3/lib/python3.9/site-packages/diffprivlib/models/logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "


LogisticRegression(accountant=BudgetAccountant(spent_budget=[(4.0, 0)]),
                   data_norm=30.479211207303948, epsilon=4.0, random_state=0)

In [22]:
dp2_lr = dp2_clf.score(X_test_scaled, y_test)
print("Differentially private LR model test accuracy (epsilon=%.2f): %.2f%%" % 
     (dp2_clf.epsilon, (dp2_lr * 100)))

Differentially private LR model test accuracy (epsilon=4.00): 66.60%


### 3. Baseline Multi-Layer Percepton

In [25]:
def evaluate_model_accuracy(model, X_test_tensor, y_test_tensor):
    """
    Evaluate the accuracy of a model on a test dataset.

    This function takes a PyTorch model, a tensor of test inputs, and a tensor of test labels,
    and returns the accuracy of the model on the test data. The model's output is assumed to be
    a probability, and the output is thresholded at 0.5 to get the predicted labels.

    Parameters:
    model (torch.nn.Module): The PyTorch model to evaluate.
    X_test_tensor (torch.Tensor): A tensor containing the test inputs.
    y_test_tensor (torch.Tensor): A tensor containing the true labels for the test data.

    Returns:
    float: The accuracy of the model on the test data, as a float between 0 and 1.
    """
    with torch.no_grad():
        model.eval()
        y_pred_prob = model(X_test_tensor)
        y_pred_test = (y_pred_prob >= 0.5).float()
        accuracy = (y_pred_test == y_test_tensor).float().mean()
    return accuracy

In [26]:
baseline_mlp = evaluate_model_accuracy(model, X_test_tensor, y_test_tensor)
print("Non-private MLP model test accuracy: %.2f%%" % (baseline_mlp.item() * 100))

Non-private MLP model test accuracy: 70.49%


### 4. Differentially Private Multi-Layer Perceptron using DP-SGD

In [27]:
model = MLPModel(input_size, hidden_size1, hidden_size2,hidden_size3, output_size)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [28]:
# Define PrivacyEngine parameters
DELTA = 1e-5
EPSILON = 8.0
EPOCHS = 10
MAX_GRAD_NORM = 1.0

In [29]:
# Convert data to PyTorch DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [30]:
def train_model_with_privacy(model, criterion, optimizer, train_loader, num_epochs, DELTA, EPSILON, EPOCHS, MAX_GRAD_NORM):
    """
    Train a PyTorch model with differential privacy.

    This function wraps the model with a PrivacyEngine, which modifies the optimizer to enforce differential privacy.
    It then trains the model for a specified number of epochs, and returns the trained model.

    Parameters:
    model (torch.nn.Module): The PyTorch model to train.
    criterion (torch.nn.Module): The loss function.
    optimizer (torch.optim.Optimizer): The optimizer.
    train_loader (torch.utils.data.DataLoader): The data loader for the training data.
    num_epochs (int): The number of epochs to train for.
    DELTA (float): The target_delta parameter for the PrivacyEngine.
    EPSILON (float): The target_epsilon parameter for the PrivacyEngine.
    EPOCHS (int): The epochs parameter for the PrivacyEngine.
    MAX_GRAD_NORM (float): The max_grad_norm parameter for the PrivacyEngine.

    Returns:
    torch.nn.Module: The trained model.
    """
    # Wrap the model with PrivacyEngine
    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        target_delta=DELTA,
        target_epsilon=EPSILON, 
        epochs=EPOCHS,
        max_grad_norm=MAX_GRAD_NORM,
    )

    # Training loop
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

    return model

In [31]:
model = train_model_with_privacy(model, criterion, optimizer, train_loader, num_epochs=3,
                                 DELTA=DELTA, EPSILON=EPSILON, EPOCHS=EPOCHS, MAX_GRAD_NORM=MAX_GRAD_NORM)

/Users/keshvigupta/opt/anaconda3/lib/python3.9/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/Users/keshvigupta/opt/anaconda3/lib/python3.9/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/Users/keshvigupta/opt/anaconda3/lib/python3.9/site-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
/Users/keshvigupta/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed 

Epoch [1/3], Loss: 0.6880483031272888
Epoch [2/3], Loss: 0.6666591167449951
Epoch [3/3], Loss: 0.6233127117156982


In [32]:
dp_mlp = evaluate_model_accuracy(model, X_test_tensor, y_test_tensor)
print("Differentially private MLP model test accuracy (epsilon=%.2f): %.2f%%" % 
      (EPSILON, (dp_mlp.item() * 100)))

Differentially private MLP model test accuracy (epsilon=8.00): 65.14%


In [33]:
# Define new PrivacyEngine parameters for increased privacy
DELTA = 1e-5
EPSILON = 4.0  # Decrease epsilon for more privacy
EPOCHS = 10
MAX_GRAD_NORM = 0.5  # Decrease max_grad_norm for more privacy

In [34]:
model = MLPModel(input_size, hidden_size1, hidden_size2,hidden_size3, output_size)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
model2 = train_model_with_privacy(model, criterion, optimizer, train_loader, num_epochs=3,
                                 DELTA=DELTA, EPSILON=EPSILON, EPOCHS=EPOCHS, MAX_GRAD_NORM=MAX_GRAD_NORM)

Epoch [1/3], Loss: 0.6738815307617188
Epoch [2/3], Loss: 0.6914243698120117
Epoch [3/3], Loss: 0.6762195229530334


In [35]:
dp2_mlp = evaluate_model_accuracy(model, X_test_tensor, y_test_tensor)
print("Differentially private MLP model test accuracy (epsilon=%.2f): %.2f%%" % 
      (EPSILON, (dp2_mlp.item() * 100)))

Differentially private MLP model test accuracy (epsilon=4.00): 54.98%
